# Intra model comparison


1. get examples that the model gets wrong at latest epoch vs those that the model gets correct in the first epoch (or the checkpoint with the best cJGA for that metric) 
1. calculate frequencies of these examples across different seed values. 

- one function that finds the final checkpoint 
- one function that finds the checkpoint with the best cJGA on a perturbation 
- one function that takes the two files from the previous steps and finds examples (also useful for inter model comparison)

In [36]:
%load_ext autoreload
%autoreload 2

In [48]:
import os 
from pathlib import Path 
import json 
from qualitative_analysis_utils import * 

In [38]:
aug = "NEI"
model_dir = "/data/home/justincho/ParlAI/models/bart_scratch_multiwoz2.3/fs_False_prompts_True_lr5e-05_bs4_uf1_sd0/"

In [40]:
logs = list(Path(model_dir).glob(f"*{aug}_world_logs*.jsonl"))
test_reports = list(Path(model_dir).glob(f"*{aug}_report*.json"))
len(logs), len(test_reports)

logs = sort_by_steps(logs)
test_reports = sort_by_steps(test_reports)
# logs, test_reports

In [44]:
final_log = logs[-1]

In [45]:
max_cjga_log, max_cjga = find_checkpoint_with_best_cjga(logs, test_reports)
assert max_cjga_log != final_log

In [50]:
compare_two_logs(final_log, max_cjga_log)

In [52]:
with open(str(final_log), "r") as f: 
    data = f.readlines() 
    json_data = [json.loads(l) for l in data]

In [53]:
json_data[0]['dialog'][0][1]['text'].split(",")

['restaurant food italian',
 ' restaurant pricerange cheap',
 ' restaurant name ask restaurant',
 ' restaurant area centre',
 ' hotel people 6',
 ' hotel day sunday',
 ' hotel stay 4',
 ' hotel area dontcare',
 ' hotel parking yes',
 ' hotel pricerange cheap',
 ' hotel internet yes',
 ' hotel type guesthouse',
 '']

In [57]:
def get_dial_id_from_log_line(log_line): 
    curr = log_line['dialog'][0][0]
    
    dial_id = f"{curr['dial_id']}-{curr['turn_num']}"

    return dial_id 

def get_dialogue_state(): 

    return 

def get_dict_diff(): 

    return 

df_list = [] 
from tqdm import tqdm

for idx in tqdm(range(0, len(json_data), 2)):
    original = json_data[idx]
    perturbed = json_data[idx+1]

    orig_label = original['dialog'][0][0]
    orig_pred = original['dialog'][0][1]
    aug_label = perturbed['dialog'][0][0]
    aug_pred = perturbed['dialog'][0][1]

    original_dial_id = get_dial_id_from_log_line(original)
    perturbed_dial_id = get_dial_id_from_log_line(perturbed)
    assert original_dial_id == perturbed_dial_id

    orig_text = orig_label['text']
    aug_text = aug_label['text']

    orig_metrics = orig_pred['metrics']
    aug_metrics = aug_pred['metrics']

    orig_gt_dst = orig_label['eval_labels']
    orig_pred_dst = orig_pred['text']

    aug_gt_dst = aug_label['eval_labels']
    aug_pred_dst = aug_pred['text']

    jga_original = orig_metrics['jga_original']
    jga_perturbed = aug_metrics['jga_perturbed']
    jga_conditional = aug_metrics['jga_conditional']

    print(aug_gt_dst)
    print(aug_pred_dst)
    print(orig_gt_dst)
    print(orig_pred_dst)

    df_list.append(
        {
            "dialid": original_dial_id, 
            "orig_text": orig_text, 
            "perturbed_text": aug_text, 
            "orig_label": orig_gt_dst, 
            "perturbed_label": aug_gt_dst, 
            "orig_pred": orig_pred_dst, 
            "perturbed_pred": aug_pred_dst, 
            "jga_orig": jga_original, 
            "jga_perturbed": jga_perturbed, 
            "jga_conditional": jga_conditional 
        }
    )


    # break 


100%|██████████| 5827/5827 [00:00<00:00, 78056.05it/s]


In [65]:
json_data[0]['dialog'][0][1]['metrics']['jga_conditional']

KeyError: 'jga_conditional'

In [ ]:
# format all jsonl files to have the following values per row
columns = ["dialid", "orig_text", "perturbed_text", "orig_label", "perturbed_label", "orig_pred", "perturbed_pred", "jga_orig", "jga_perturbed", "jga_conditional"] 
# then for two different checkpoints, find the cases where earlier checkpoint got it correct consistently, but didn't in the later checkpoint 

# run code to find patterns and print statistics for these patterns 

In [ ]:
# eval